## Looking at Few Favorite Rules

### SQLAssessmentFavorites.ipynb

Written  by:

Taiob Ali

Last Modified: Sept 7 2022

In [ ]:
# Uncomment and run Install-Module only the first time 
# Install-Module -Name SqlServer -AllowClobber -Force
Import-Module -Name SqlServer
Get-Module -ListAvailable -Name Sqlserver

In [ ]:
# Declare variables
$serverInstance = Get-SqlInstance -ServerInstance 'localhost\SQL2019'
$database = Get-SqlDatabase -ServerInstance 'localhost\SQL2019' -Name SqlAssessmentDemo
$samplesPath='C:\SQLAsmnt\CustomizationSamples'

In [ ]:
# Run a rule by its id (DeprecatedFeatures)
Invoke-SqlAssessment $serverInstance -Check DeprecatedFeatures

```
--Change one of the tempdb file size
USE [master];
GO
ALTER DATABASE [tempdb] MODIFY FILE ( NAME = N'temp8', SIZE = 10 );
GO
--Revert
USE [tempdb];
GO
DBCC SHRINKFILE (N'temp8' , 8);
GO
```

In [ ]:
# Run a rule by its id (TempDBFilesNotSameSize)
Invoke-SqlAssessment $serverInstance -Check TempDBFilesNotSameSize

```
--Create a login with same password
USE [master];
GO
IF EXISTS 
  (SELECT 
    name  
   FROM master.sys.server_principals
   WHERE name = 'nesqlugdemo')
BEGIN
DROP LOGIN [nesqlugdemo];
END
CREATE LOGIN [nesqlugdemo] WITH PASSWORD=N'nesqlugdemo', DEFAULT_DATABASE=[master];
GO
```

In [ ]:
# Run a rule by its id (LoginEqPassword)
Invoke-SqlAssessment $serverInstance -Check 'LoginEqPassword' -Configuration $(join-path $samplesPath "EnableLoginEqPassword.json")

```
/*
  Create sample table and indexes
  Copied form https://www.mssqltips.com/sqlservertip/3604/identify-sql-server-indexes-with-duplicate-columns/
*/
USE [SqlAssessmentDemo];
GO
DROP TABLE  IF EXISTS testtable1;
GO
CREATE TABLE testtable1 
(
  [col1] [int] NOT NULL primary key  clustered,
  [col2] [int]  NULL,
  [col3] [int]  NULL,
  [col4] [varchar](50) NULL
); 

CREATE INDEX idx_testtable1_col2col3 on testtable1 (col2  asc, col3 asc);
CREATE INDEX idx_testtable1_col2col4 on testtable1 (col2  asc, col4 asc);
CREATE INDEX idx_testtable1_col3 on testtable1 (col3  asc);
CREATE INDEX idx_testtable1_col3col4 on testtable1 (col3  asc, col4 asc);
GO

DROP TABLE  IF EXISTS testtable2  ;
GO
CREATE TABLE testtable2 
(
  [col1] [int] NOT NULL primary key  clustered,
  [col2] [int]  NULL,
  [col3] [int]  NULL,
  [col4] [varchar](50) NULL
); 
 
CREATE INDEX idx_testtable2_col3col4 on testtable2 (col3  asc, col4 asc);
CREATE INDEX idx_testtable2_col3col4_1 on testtable2 (col3  asc, col4 asc);
```

In [ ]:
# Run a rule by its id (DuplicateIndexes)
Invoke-SqlAssessment $database -Check DuplicateIndexes

In [ ]:
# Run a rule by its id (UnusedIndex)
Invoke-SqlAssessment $database -Check UnusedIndex

Setting the 'max server memory' value too high can cause a single instance of SQL Server to compete for memory with other SQL Server instances hosted on the same machine. However, setting this value too low could cause significant memory pressure and performance problems.

```
--Adjust Max Memory
USE [master];
GO
EXEC sys.sp_configure N'max server memory (MB)', N'32000';
GO
RECONFIGURE WITH OVERRIDE;
GO
--Revert
EXEC sys.sp_configure N'max server memory (MB)', N'28000';
GO
RECONFIGURE WITH OVERRIDE;
GO

```

In [ ]:
# Run a rule by its id (MaxMemory)
Invoke-SqlAssessment $serverInstance -Check MaxMemory

In [ ]:
Invoke-SqlAssessment $serverInstance -Check PowerPlanSchemeNotRecommended

In [ ]:
# list all available power plans
powercfg /list

In [ ]:
# Switch power plan to 'High Performance'
# Run the "PowerPlanSchemeNotRecommended" check again
powercfg.exe -SETACTIVE 8c5e7fda-e8bf-4a96-9a85-a6e23a8c635c

Revert to balance power plan for future demo

In [ ]:
# Switch power plan to 'Balanced'
powercfg.exe -SETACTIVE 381b4222-f694-41f0-9685-ff5bb260df2e

Database files have growth ratio in percentage

In [ ]:
Invoke-SqlAssessment $database -Check PercentAutogrows